# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
#import matplotlib.pyplot as pltimport gmaps
import numpy as np
import pandas as pd
import requests
import time
from matplotlib import pyplot as plt
import pprint
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
from matplotlib import pyplot as plt
import pprint

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../WeatherPy/output_data/Weather_Data.csv")
weather_df = pd.DataFrame(weather_data)
weather_df

,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,Narok,-1.08,35.86,54.07,94,46,4.61
1,Puerto Ayora,-0.74,-90.35,75.00,90,71,1.99
2,Nanortalik,60.14,-45.24,45.27,96,99,5.01
3,Bambous Virieux,-20.34,57.76,72.73,78,75,12.75
4,Necochea,-38.55,-58.74,48.00,88,100,3.00
...,...,...,...,...,...,...,...
495,Kieta,-6.22,155.63,84.42,73,18,3.47
496,Codrington,-38.27,141.97,53.76,63,92,10.16
497,Nago,26.59,127.98,84.13,83,75,16.11
498,Zaysan,47.47,84.87,74.26,39,16,10.42


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Generate Heat Map Based on Humidity Percentages

# Establish API Key
gmaps.configure(api_key=g_key)

# Define Heat Map Locations and Weights
locations = weather_df[["Latitude", "Longitude"]].astype(float)
humidity = weather_df["Humidity (%)"].astype(float)

# Create Heat Map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=True, max_intensity=100,
                                point_radius = 7)
fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

In [4]:
# Narrow Down Personal Optimal Weather Conditions
optimal_weather = weather_df[(weather_df["Temperature (F)"] > 65) & (weather_df["Temperature (F)"] < 75)]
optimal_weather = optimal_weather[(optimal_weather["Humidity (%)"] < 50)]

# Create New DataFrame to Store Hotel Data
optimal_df = optimal_weather[["City", "Latitude", "Longitude"]].reset_index(drop=True)
optimal_df["Hotel Name"] = ""
optimal_df["Hotel Address"] = ""
optimal_df["Hotel Latitude"] = ""
optimal_df["Hotel Longitude"] = ""
optimal_df


,City,Latitude,Longitude,Hotel Name,Hotel Address,Hotel Latitude,Hotel Longitude
0,Vitim,59.45,112.56,,,,
1,Medicine Hat,50.05,-110.67,,,,
2,Pacific Grove,36.62,-121.92,,,,
3,Port Shepstone,-30.74,30.45,,,,
4,Bundaberg,-24.85,152.35,,,,
5,Sioux Lookout,50.10,-91.92,,,,
6,Namioka,40.71,140.59,,,,
7,Fort Frances,48.60,-93.40,,,,
8,Hulan Ergi,47.20,123.63,,,,
9,Jalu,29.03,21.55,,,,


In [5]:
# Obtain Hotel Data From Google Places API

# Establish Parameters For Hotel Search
params = {
    'types': 'lodging',
    'radius': 5000,
    "key": g_key
}

# Iterate Each City Location to Obtain Hotel Data
for index, row in optimal_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    name_address = requests.get(base_url, params=params)
    
    name_address = name_address.json()
            
    try:
        optimal_df.loc[index, "Hotel Name"] = name_address['results'][0]['name']
        optimal_df.loc[index, "Hotel Address"] = name_address['results'][0]['vicinity']
        optimal_df.loc[index, "Hotel Latitude"] = name_address['results'][0]['geometry']['location']['lat']
        optimal_df.loc[index, "Hotel Longitude"] = name_address['results'][0]['geometry']['location']['lng']
    except (KeyError, IndexError):
        print("N/A")
        
optimal_df

N/A


,City,Latitude,Longitude,Hotel Name,Hotel Address,Hotel Latitude,Hotel Longitude
0,Vitim,59.45,112.56,Gostinitsa V Apteke,"Ленина ул, 41, Vitim",59.4438,112.575
1,Medicine Hat,50.05,-110.67,Ramada Limited,"773 8 Street Southwest, Medicine Hat",50.0312,-110.705
2,Pacific Grove,36.62,-121.92,Lovers Point Inn,"625 Ocean View Boulevard, Pacific Grove",36.6256,-121.917
3,Port Shepstone,-30.74,30.45,My Den Beachfront B&B And Self-Catering,"3 North Road, Oslo Beach, Port Shepstone",-30.7612,30.4455
4,Bundaberg,-24.85,152.35,Takalvan Motel,"11 Takalvan Street, Bundaberg West",-24.8723,152.334
5,Sioux Lookout,50.10,-91.92,Knobby's Fly-In Camps,"25 Wellington Street, Sioux Lookout",50.093,-91.9121
6,Namioka,40.71,140.59,Restaurant & INN Sunrise,"Nishitanemoto-13-1 Namioka Oaza Megasawa, Aomori",40.7058,140.58
7,Fort Frances,48.60,-93.40,LA PLACE RENDEZ-VOUS,"1201 Idylwild Main, Fort Frances",48.6212,-93.3594
8,Hulan Ergi,47.20,123.63,Qiqihaer Zhongtian Hotel,"No. 179, Heping Street, CN Heilongjiang Qiqiha...",47.2131,123.62
9,Jalu,29.03,21.55,Jalu Hotel,Jalu,29.0327,21.5458


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>

</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in optimal_df.iterrows()]
locations = optimal_df[["Latitude", "Longitude"]]

In [21]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations,
    info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))